In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from datetime import datetime, timedelta
import time
import numpy as np
import re
from dateutil.relativedelta import relativedelta
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from scipy import stats

In [2]:
crash = pd.read_csv('crash_clean.csv')

In [3]:
crash.head()

,Unnamed: 0,Crash_ID,Crash_Fatal_Fl,Crash_Date,Crash_Time,Rpt_Rdwy_Sys_ID,Rpt_Hwy_Num,Private_Dr_Fl,Crash_Speed_Limit,At_Intrsct_Fl,...,Obj_Struck_ID,Latitude,Longitude,Street_Name,Street_Nbr,Street_Name_2,Crash_Sev_ID,Day_of_Week,Tot_Injry_Cnt,Death_Cnt
0,0,11152580,N,01/15/2010,12:06 PM,19,NaN,N,35,N,...,64,30.376001,-97.673639,E BRAKER LN,867.0,NaN,3,FRI,2,0
1,1,11153849,N,01/15/2010,04:32 PM,19,NaN,N,0,Y,...,29,30.307715,-97.726472,E 45TH ST,300.0,UNKNOWN,3,FRI,1,0
2,2,11154056,N,01/13/2010,06:38 PM,19,NaN,N,35,Y,...,64,30.265225,-97.740032,E 4TH ST,398.0,TRINITY ST,3,WED,1,0
3,3,11154058,N,01/13/2010,03:47 PM,19,NaN,N,30,N,...,64,30.300115,-97.732672,E 38TH ST,200.0,UNKNOWN,3,WED,1,0
4,4,11154059,N,01/12/2010,04:51 PM,1,35,N,55,N,...,64,30.292341,-97.721250,IH0035,NaN,NaN,5,TUE,0,0


In [4]:
primary_person = pd.read_csv('primary_person_clean.csv')

In [5]:
p_person=primary_person[primary_person['Prsn_Occpnt_Pos_ID']==1]

In [6]:
p_person.head(100)

,Unnamed: 0,Crash_ID,Prsn_Occpnt_Pos_ID,Prsn_Injry_Sev_ID,Prsn_Age,Prsn_Ethnicity_ID,Prsn_Gndr_ID,Prsn_Ejct_ID,Prsn_Rest_ID,Prsn_Airbag_ID,Prsn_Helmet_ID,Prsn_Alc_Spec_Type_ID,Prsn_Alc_Rslt_ID,Prsn_Bac_Test_Rslt,Prsn_Drg_Rslt_ID,Drvr_Drg_Cat_1_ID
0,0,11152580,1.0,3.0,24.0,1.0,2.0,1.0,1.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0
1,1,11152580,1.0,5.0,60.0,2.0,1.0,1.0,1.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0
2,2,11152580,1.0,5.0,23.0,2.0,1.0,1.0,1.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0
3,3,11153849,1.0,5.0,NaN,2.0,1.0,1.0,1.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0
4,4,11153849,1.0,5.0,NaN,2.0,1.0,1.0,1.0,3.0,97.0,4.0,NaN,NaN,97.0,97.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,97,11161443,1.0,5.0,52.0,2.0,2.0,1.0,1.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0
98,98,11161443,1.0,3.0,48.0,1.0,2.0,1.0,1.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0
99,99,11161802,1.0,5.0,21.0,1.0,2.0,1.0,1.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0
100,100,11161803,1.0,5.0,82.0,1.0,2.0,1.0,1.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0


In [7]:
merged = crash.merge(p_person, left_on='Crash_ID', right_on='Crash_ID')

In [8]:
merged.head(100)

,Unnamed: 0_x,Crash_ID,Crash_Fatal_Fl,Crash_Date,Crash_Time,Rpt_Rdwy_Sys_ID,Rpt_Hwy_Num,Private_Dr_Fl,Crash_Speed_Limit,At_Intrsct_Fl,...,Prsn_Gndr_ID,Prsn_Ejct_ID,Prsn_Rest_ID,Prsn_Airbag_ID,Prsn_Helmet_ID,Prsn_Alc_Spec_Type_ID,Prsn_Alc_Rslt_ID,Prsn_Bac_Test_Rslt,Prsn_Drg_Rslt_ID,Drvr_Drg_Cat_1_ID
0,0,11152580,N,01/15/2010,12:06 PM,19,NaN,N,35,N,...,2.0,1.0,1.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0
1,0,11152580,N,01/15/2010,12:06 PM,19,NaN,N,35,N,...,1.0,1.0,1.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0
2,0,11152580,N,01/15/2010,12:06 PM,19,NaN,N,35,N,...,1.0,1.0,1.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0
3,1,11153849,N,01/15/2010,04:32 PM,19,NaN,N,0,Y,...,1.0,1.0,1.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0
4,1,11153849,N,01/15/2010,04:32 PM,19,NaN,N,0,Y,...,1.0,1.0,1.0,3.0,97.0,4.0,NaN,NaN,97.0,97.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,50,11161443,N,01/22/2010,07:08 AM,2,183,N,70,N,...,2.0,1.0,1.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0
96,50,11161443,N,01/22/2010,07:08 AM,2,183,N,70,N,...,2.0,1.0,1.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0
97,51,11161802,N,01/05/2010,08:17 AM,19,NaN,N,30,N,...,2.0,1.0,1.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0
98,52,11161803,N,01/05/2010,10:59 AM,19,NaN,N,45,N,...,2.0,1.0,1.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0


In [9]:
#Export merged file to csv before transformations to have a backup
#merged.to_csv('demographic.csv')

In [10]:
au_pop = pd.read_csv('AustinPop.csv')

In [11]:
au_pop.head()

,Year,Population
0,2010,806423
1,2011,828694
2,2012,854841
3,2013,875463
4,2014,901734


In [12]:
merged["Crash_Date"] = pd.to_datetime(merged["Crash_Date"], format= "%m/%d/%Y")

merged["time_segment"] = np.where((merged["Crash_Date"] <= pd.Timestamp(2014,6,1)),"before", 
                              np.where((merged["Crash_Date"] > pd.Timestamp(2014,6,1)) & (merged["Crash_Date"] <= pd.Timestamp(2016,5,10)), "entry",
                                       np.where((merged["Crash_Date"] > pd.Timestamp(2016,5,10)) & (merged["Crash_Date"] <= pd.Timestamp(2017,5,29)), "exit",
                                                np.where((merged["Crash_Date"] > pd.Timestamp(2017,5,29)), "re-entry",
                                                         "error"))))
merged["uber_present"] = np.where((merged["Crash_Date"] <= pd.Timestamp(2014,6,1)),0, 
                              np.where((merged["Crash_Date"] > pd.Timestamp(2014,6,1)) & (merged["Crash_Date"] <= pd.Timestamp(2016,5,10)), 1,
                                       np.where((merged["Crash_Date"] > pd.Timestamp(2016,5,10)) & (merged["Crash_Date"] <= pd.Timestamp(2017,5,29)), 0,
                                                np.where((merged["Crash_Date"] > pd.Timestamp(2017,5,29)), 1,
                                                         "error"))))

In [13]:
merged

,Unnamed: 0_x,Crash_ID,Crash_Fatal_Fl,Crash_Date,Crash_Time,Rpt_Rdwy_Sys_ID,Rpt_Hwy_Num,Private_Dr_Fl,Crash_Speed_Limit,At_Intrsct_Fl,...,Prsn_Rest_ID,Prsn_Airbag_ID,Prsn_Helmet_ID,Prsn_Alc_Spec_Type_ID,Prsn_Alc_Rslt_ID,Prsn_Bac_Test_Rslt,Prsn_Drg_Rslt_ID,Drvr_Drg_Cat_1_ID,time_segment,uber_present
0,0,11152580,N,2010-01-15,12:06 PM,19,NaN,N,35,N,...,1.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0,before,0
1,0,11152580,N,2010-01-15,12:06 PM,19,NaN,N,35,N,...,1.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0,before,0
2,0,11152580,N,2010-01-15,12:06 PM,19,NaN,N,35,N,...,1.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0,before,0
3,1,11153849,N,2010-01-15,04:32 PM,19,NaN,N,0,Y,...,1.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0,before,0
4,1,11153849,N,2010-01-15,04:32 PM,19,NaN,N,0,Y,...,1.0,3.0,97.0,4.0,NaN,NaN,97.0,97.0,before,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276802,11531,17279716,N,2019-08-25,05:57 PM,19,NaN,N,15,Y,...,1.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0,re-entry,1
276803,11532,17279722,N,2019-08-21,01:22 PM,19,NaN,N,30,Y,...,10.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0,re-entry,1
276804,11532,17279722,N,2019-08-21,01:22 PM,19,NaN,N,30,Y,...,10.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0,re-entry,1
276805,11533,17279773,N,2019-09-04,06:30 PM,19,NaN,N,35,N,...,8.0,2.0,97.0,4.0,NaN,NaN,97.0,97.0,re-entry,1


merged_gb = merged[['Crash_ID','Crash_Date']]
merged_gb = merged_gb.set_index('Crash_Date')
annual_crash_numbers = merged_gb.resample('Y').count()
annual_crash_numbers = annual_crash_numbers.reset_index()

In [14]:
merged['Hour'] = merged['Crash_Time'].apply(lambda x: x.split(':')[0])
merged['AMPM'] = merged['Crash_Time'].apply(lambda x: x.split(' ')[-1])

In [15]:
merged["age_bracket"] = np.where((merged["Prsn_Age"] <= 17),"less than 17", 
                              np.where((merged["Prsn_Age"] > 17) & (merged["Prsn_Age"] <= 24), "18-24",
                                       np.where((merged["Prsn_Age"] > 24) & (merged["Prsn_Age"] <= 34), "25-34",
                                                np.where((merged["Prsn_Age"] > 34) & (merged["Prsn_Age"] <= 44), "35-44",
                                                        np.where((merged["Prsn_Age"] > 44), "45+","error")))))

In [16]:
merged["Crash_Time_24H"] = pd.to_datetime(merged["Crash_Time"]).dt.strftime('%H')
merged["Crash_Time_24H"] = merged["Crash_Time_24H"].astype(float)



In [17]:
merged["time_bracket"] = np.where((merged["Crash_Time_24H"] <= 3),"night", 
                              np.where((merged["Crash_Time_24H"] > 3) & (merged["Crash_Time_24H"] <= 9), "morning",
                                       np.where((merged["Crash_Time_24H"] > 9) & (merged["Crash_Time_24H"] <= 15), "day",
                                                np.where((merged["Crash_Time_24H"] > 15) & (merged["Crash_Time_24H"] <= 20), "afternoon",
                                                        np.where((merged["Crash_Time_24H"] > 20), "night","error")))))

In [18]:
merged

,Unnamed: 0_x,Crash_ID,Crash_Fatal_Fl,Crash_Date,Crash_Time,Rpt_Rdwy_Sys_ID,Rpt_Hwy_Num,Private_Dr_Fl,Crash_Speed_Limit,At_Intrsct_Fl,...,Prsn_Bac_Test_Rslt,Prsn_Drg_Rslt_ID,Drvr_Drg_Cat_1_ID,time_segment,uber_present,Hour,AMPM,age_bracket,Crash_Time_24H,time_bracket
0,0,11152580,N,2010-01-15,12:06 PM,19,NaN,N,35,N,...,NaN,97.0,97.0,before,0,12,PM,18-24,12.0,day
1,0,11152580,N,2010-01-15,12:06 PM,19,NaN,N,35,N,...,NaN,97.0,97.0,before,0,12,PM,45+,12.0,day
2,0,11152580,N,2010-01-15,12:06 PM,19,NaN,N,35,N,...,NaN,97.0,97.0,before,0,12,PM,18-24,12.0,day
3,1,11153849,N,2010-01-15,04:32 PM,19,NaN,N,0,Y,...,NaN,97.0,97.0,before,0,04,PM,error,16.0,afternoon
4,1,11153849,N,2010-01-15,04:32 PM,19,NaN,N,0,Y,...,NaN,97.0,97.0,before,0,04,PM,error,16.0,afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276802,11531,17279716,N,2019-08-25,05:57 PM,19,NaN,N,15,Y,...,NaN,97.0,97.0,re-entry,1,05,PM,18-24,17.0,afternoon
276803,11532,17279722,N,2019-08-21,01:22 PM,19,NaN,N,30,Y,...,NaN,97.0,97.0,re-entry,1,01,PM,25-34,13.0,day
276804,11532,17279722,N,2019-08-21,01:22 PM,19,NaN,N,30,Y,...,NaN,97.0,97.0,re-entry,1,01,PM,25-34,13.0,day
276805,11533,17279773,N,2019-09-04,06:30 PM,19,NaN,N,35,N,...,NaN,97.0,97.0,re-entry,1,06,PM,18-24,18.0,afternoon


In [19]:
#merged.to_csv('demographic.csv')

Analyzing crashes/people at different time periods

In [20]:
#Set up time_segment # of months

cols = ["before", "entry", "exit", "re-entry"]
seg_months = [52.5, 23, 12.5, 27.5]

df_time_segment_months = pd.DataFrame([seg_months])
df_time_segment_months.columns =cols
df_time_segment_months = df_time_segment_months.T
df_time_segment_months.columns = ["months_in_segment"]
df_time_segment_months["time_segment"] = df_time_segment_months.index
df_time_segment_months.head()

,months_in_segment,time_segment
before,52.5,before
entry,23.0,entry
exit,12.5,exit
re-entry,27.5,re-entry


In [21]:
#import population for normalization
df_pop = pd.read_csv("AustinMonthlyPop.csv")
df_pop.rename(columns={"Unnamed: 1":"Month"}, inplace=True)
df_pop.head()

,Year,Month,Population
0,2010,1,797593
1,2010,2,799287
2,2010,3,800998
3,2010,4,802767
4,2010,5,804584


In [22]:
#Merge dataframe with population and normalize
df_people = merged[["Crash_ID","Crash_Fatal_Fl","Crash_Sev_ID","Prsn_Occpnt_Pos_ID","Prsn_Injry_Sev_ID","Prsn_Age", "Prsn_Ethnicity_ID","Prsn_Gndr_ID", "uber_present", "age_bracket", "Crash_Time_24H", "time_bracket","Day_of_Week","time_segment","Crash_Date", "Crash_Time"]].copy()
df_people["Month"] = pd.DatetimeIndex(df_people['Crash_Date']).month
df_people["Year"] = pd.DatetimeIndex(df_people['Crash_Date']).year
df_people = df_people.merge(df_pop, on=["Year", "Month"], how="left")
df_people["Crash_Norm"] = 1/(df_people["Population"]/10**3) #Convert to crashes per thousand people

#Merge df with segmentation before, entry, exit, re-entry 
df_people = df_people.merge(df_time_segment_months, on="time_segment", how="left")

#Create a population normalized and annualized value for each crash for aggregation below
#Value is crash per 1,000 residents per months in time segment (for annualization)
df_people["Crash_Norm_Annual"] = df_people["Crash_Norm"] / (df_people["months_in_segment"]/12) #Annualize data

In [26]:
df_people["Crash_ID"].value_counts()

16186067    18
13656264    12
16814892    12
14404973    12
14601410    11
            ..
13661251     1
14410801     1
11738158     1
16556578     1
15204352     1
Name: Crash_ID, Length: 146026, dtype: int64

In [27]:
df_people

,Crash_ID,Crash_Fatal_Fl,Crash_Sev_ID,Prsn_Occpnt_Pos_ID,Prsn_Injry_Sev_ID,Prsn_Age,Prsn_Ethnicity_ID,Prsn_Gndr_ID,uber_present,age_bracket,...,Day_of_Week,time_segment,Crash_Date,Crash_Time,Month,Year,Population,Crash_Norm,months_in_segment,Crash_Norm_Annual
0,11152580,N,3,1.0,3.0,24.0,1.0,2.0,0,18-24,...,FRI,before,2010-01-15,12:06 PM,1,2010,797593,0.001254,52.5,0.000287
1,11152580,N,3,1.0,5.0,60.0,2.0,1.0,0,45+,...,FRI,before,2010-01-15,12:06 PM,1,2010,797593,0.001254,52.5,0.000287
2,11152580,N,3,1.0,5.0,23.0,2.0,1.0,0,18-24,...,FRI,before,2010-01-15,12:06 PM,1,2010,797593,0.001254,52.5,0.000287
3,11153849,N,3,1.0,5.0,NaN,2.0,1.0,0,error,...,FRI,before,2010-01-15,04:32 PM,1,2010,797593,0.001254,52.5,0.000287
4,11153849,N,3,1.0,5.0,NaN,2.0,1.0,0,error,...,FRI,before,2010-01-15,04:32 PM,1,2010,797593,0.001254,52.5,0.000287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276802,17279716,N,5,1.0,5.0,22.0,1.0,2.0,1,18-24,...,SUN,re-entry,2019-08-25,05:57 PM,8,2019,997875,0.001002,27.5,0.000437
276803,17279722,N,5,1.0,5.0,28.0,0.0,1.0,1,25-34,...,WED,re-entry,2019-08-21,01:22 PM,8,2019,997875,0.001002,27.5,0.000437
276804,17279722,N,5,1.0,5.0,27.0,0.0,1.0,1,25-34,...,WED,re-entry,2019-08-21,01:22 PM,8,2019,997875,0.001002,27.5,0.000437
276805,17279773,N,5,1.0,5.0,20.0,1.0,2.0,1,18-24,...,WED,re-entry,2019-09-04,06:30 PM,9,2019,1000112,0.001000,27.5,0.000436


In [68]:
df_people = df_people.assign(P_per_crash=df_people.groupby('Crash_ID').Prsn_Occpnt_Pos_ID.transform('sum'))
df_people['P_per_Crash_Norm_Annual'] = df_people["Crash_Norm_Annual"]/df_people["P_per_crash"]

In [69]:
df_people

,Crash_ID,Crash_Fatal_Fl,Crash_Sev_ID,Prsn_Occpnt_Pos_ID,Prsn_Injry_Sev_ID,Prsn_Age,Prsn_Ethnicity_ID,Prsn_Gndr_ID,uber_present,age_bracket,...,Crash_Date,Crash_Time,Month,Year,Population,Crash_Norm,months_in_segment,Crash_Norm_Annual,P_per_crash,P_per_Crash_Norm_Annual
0,11152580,N,3,1.0,3.0,24.0,1.0,2.0,0,18-24,...,2010-01-15,12:06 PM,1,2010,797593,0.001254,52.5,0.000287,3.0,0.000096
1,11152580,N,3,1.0,5.0,60.0,2.0,1.0,0,45+,...,2010-01-15,12:06 PM,1,2010,797593,0.001254,52.5,0.000287,3.0,0.000096
2,11152580,N,3,1.0,5.0,23.0,2.0,1.0,0,18-24,...,2010-01-15,12:06 PM,1,2010,797593,0.001254,52.5,0.000287,3.0,0.000096
3,11153849,N,3,1.0,5.0,NaN,2.0,1.0,0,error,...,2010-01-15,04:32 PM,1,2010,797593,0.001254,52.5,0.000287,2.0,0.000143
4,11153849,N,3,1.0,5.0,NaN,2.0,1.0,0,error,...,2010-01-15,04:32 PM,1,2010,797593,0.001254,52.5,0.000287,2.0,0.000143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276802,17279716,N,5,1.0,5.0,22.0,1.0,2.0,1,18-24,...,2019-08-25,05:57 PM,8,2019,997875,0.001002,27.5,0.000437,2.0,0.000219
276803,17279722,N,5,1.0,5.0,28.0,0.0,1.0,1,25-34,...,2019-08-21,01:22 PM,8,2019,997875,0.001002,27.5,0.000437,2.0,0.000219
276804,17279722,N,5,1.0,5.0,27.0,0.0,1.0,1,25-34,...,2019-08-21,01:22 PM,8,2019,997875,0.001002,27.5,0.000437,2.0,0.000219
276805,17279773,N,5,1.0,5.0,20.0,1.0,2.0,1,18-24,...,2019-09-04,06:30 PM,9,2019,1000112,0.001000,27.5,0.000436,2.0,0.000218


In [70]:
df_people.to_csv('demographics.csv')